## Assignment 3

#### Team mates: Shashank Siripragada & Sneha Swaroop, Group 4

In [56]:
import io
import PIL
import glob
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
from IPython.display import Image
from PIL import Image as pili, ImageOps as piliops
from PIL import Image, ImageFont, ImageDraw, ImageFilter
from PIL import Image as pili, ImageDraw as pild, ImageFont as pilf

Let's have a look at the dialog csv file containing English and Telugu captions for or comic book.

In [57]:
data = pd.read_csv('dialog.csv')
en_captions = data.iloc[:,0]
te_captions = data.iloc[:,1]
data.head(10)

,en,te
0,This is the beautiful place where\nthe story o...,ప్రకాశవంతమైన యుకీ కథ \nప్రారంభమయ్యే అందమైన ప్ర...
1,Yuki was an orphan.,యుకీ ఒక అనాధ.\r
2,"On a snowy winter night ten years ago, \nshe w...","పదేళ్ల క్రితం మంచుతో కూడిన \nశీతాకాలపు రాత్రి,..."
3,"Despite her unfortunate situation, \nYuki grew...","ఆమె దురదృష్టకర పరిస్థితి \nఉన్నప్పటికీ, యుకి ఒ..."
4,All right! All clean!,అయితే సరే! అన్నీ శుభ్రంగా!
5,"Hey, you!","హే, నువ్వు!"
6,Help me! Teacher!,సహాయం! టీచర్!\r
7,She often surprised \npeople with how energeti...,ఆమె ఎంత శక్తివంతమైనది అని \nఆమె తరచుగా ప్రజలను...
8,"But she had strange quirks \nas well, she'd ge...","కానీ ఆమెకి చమత్కారాలు కూడా ఉన్నాయి, \nఆమె ప్రజ..."
9,"One day her life unexpectedly \nchanged, she w...",ఒకరోజు ఆమె జీవితం ఊహించని \nవిధంగా మారిపోయింది...


#### Now we define some of the global variable such as font_size, border size, and the path containing the cartoon images.

In [58]:
font_size = 40
border = (30, 30)
border_color = 'grey'
font_color = (209, 239, 8)
TINT_COLOR = (0, 0, 0)  # Black
TRANSPARENCY = .25  # Degree of transparency, 0-100%
OPACITY = int(255 * TRANSPARENCY)

mypath = 'toons/yuki'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
imlist = glob.glob("{}/img*.png".format(mypath))

cover = pili.open('toons/yuki/label.png')
width, height = cover.size

#### Here we create a function to horizontally stack given lost of images together

In [59]:
def hstacker(input_list):
    min_shape = sorted( [(np.sum(i.size), i.size ) for i in input_list])[0][1]
    imgs_comb = np.hstack((np.asarray(i.resize(min_shape)) for i in input_list ) )
    imgs_comb = pili.fromarray(imgs_comb)
    return imgs_comb

Similar function to stack images vertically

In [65]:
def vstacker(input_list):
    min_shape = sorted([(np.sum(i.size), i.size ) for i in input_list])[0][1]
    imgs_comb = np.vstack((np.asarray( i.resize(min_shape) ) for i in input_list))
    imgs_comb = pili.fromarray( imgs_comb)
    return imgs_comb

#### Through this function we add border and caption to the given an image.

This function is capable of adding unicode fonts as well through the keywork argument **unicode**

In [66]:
def add_border_captions(img, caption, unicode=False):
    img = piliops.expand(pili.open(img), border=border,fill=border_color).convert('RGBA')
    overlay = pili.new('RGBA', img.size, TINT_COLOR+(0,))
    draw = pild.Draw(overlay)
    
    if unicode == True:
        font = pilf.truetype("c:/windows/fonts/nirmalas.ttf", font_size)
    else:
        font = pilf.truetype("arial.ttf", font_size)
    
    w, h = font.getsize(caption)
    num_lines = len(caption.split('\n'))
    x, y = 50, img.height - (num_lines)*1.5*h
    
    draw.rectangle((0, y, img.width, y + (num_lines)*1.5*h), fill=TINT_COLOR+(OPACITY,))
    draw.text((x, y), caption, fill=font_color, font=font)
    
    img = pili.alpha_composite(img, overlay)
    img = img.convert("RGB")
    return img

#### Now we tie all the functionalities together with the fuction below, where it adds the stacked comics to generate comic book pdf with the desired images. 

In [67]:
from fpdf import FPDF
pdf_english = FPDF(unit = "pt", format = [width, height])
pdf_telugu = FPDF(unit = "pt", format = [width, height])

def generate_pdf(imlist, en_captions, te_captions):
    label = 'toons/yuki/label.png'           
    pdf_english.add_page()
    pdf_english.image(label, 0, 0, width, height)
    pdf_telugu.add_page()
    pdf_telugu.image(label, 0, 0, width, height)
    
    while len(imlist) > 0:
        lines, num_cols, old_num_cols = 0, 0, 0
        cpt_begin = 1
        while lines < 14 and len(imlist) > 0:
            lines += 1
            while num_cols == old_num_cols:
                num_cols = np.random.randint(2, 4)
            old_num_cols = num_cols
            window = imlist[:num_cols]
            cpt_end = cpt_begin + num_cols-1
            
            en_window_captions, te_window_captions = en_captions[cpt_begin-1:cpt_end+1], te_captions[cpt_begin-1:cpt_end+1]
            
            en_window = [add_border_captions(i, encp) for i, encp in zip(window, en_window_captions)]
            te_window = [add_border_captions(i, tecp, unicode=True) for i, tecp in zip(window, te_window_captions)]
            
            en_merged = hstacker(en_window)
            te_merged = hstacker(te_window)
            
            en_out_fname = 'toons/temp/en{}.png'.format(lines)
            te_out_fname = 'toons/temp/te{}.png'.format(lines)
            
            en_merged.save(en_out_fname)
            te_merged.save(te_out_fname)
            
            cpt_begin = cpt_end+1
            imlist = imlist[num_cols:]

    
    for i in range(1, lines, 3):
        en_inputs = [pili.open('toons/temp/en{}.png'.format(n)) for n in range(i, i+3)]
        te_inputs = [pili.open('toons/temp/te{}.png'.format(n)) for n in range(i, i+3)]
        en_stacked  = vstacker(en_inputs)
        te_stacked  = vstacker(te_inputs)
        en_out = 'toons/temp/env{}.png'.format(i)
        te_out = 'toons/temp/tev{}.png'.format(i)
            
        en_stacked.save(en_out)
        te_stacked.save(te_out)
        pdf_english.add_page()
        pdf_english.image(en_out, 0, 0, width, height)
        pdf_telugu.add_page()
        pdf_telugu.image(te_out, 0, 0, width, height)
        
generate_pdf(imlist, en_captions, te_captions)
        
pdf_english.output("Yuki_English.pdf", "F")
pdf_telugu.output("Yuki_Telugu.pdf", "F")

print("Your comicbooks are ready!")




<ipython-input-59-4e6b642a4a8f>:3: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  imgs_comb = np.hstack((np.asarray(i.resize(min_shape)) for i in input_list ) )


Your comicbooks are ready!
